# **<center> Análisis de Sentimientos de los candidatos presidenciales </br> Ecuador 2021, en la red social Twitter </center>**
<h6>
    <center> Miguel Angel Macias, Jonnathan Campoberde <br/> 
    <i>Universidad de Cuenca <br/>  <i>Facultad de Ingenieria - Escuela de Sistemas<br/> Cuenca, Ecuador</i>
    <br/> { mangel.maciasn, jonnathan.campoberde }@ucuenca.edu.ec</center>
<h6>

# **I.&emsp;Introducción**
El presente trabajo tiene como objetivo emplear de manera práctica todos los conocimientos adquiridos en el transcurso de la asignatura “Text Mining”. Se trabajará con un dataset... . Se busca analizar el ..., detectando las ... por medio de ... . Todo esto con el propósito de ... .

## **1.1.&emsp;Objetivos del Proyecto**
>El objetivo del proyecto es el Análisis de Sentimientos de los candidatos presidenciales de Ecuador 2021, para luego contrastar los resultados obtenidos con los resultados oficiales de las elecciones Posteriormente se desea responder a las preguntas definidas en 1.2.

## **1.2.&emsp;Preguntas de investigación**
>- RQ1:     ¿Qué tan precisamente los datos de Twitter reflejan el sentimiento político en un proceso electoral?
>- RQ1:     ¿En qué grado de aceptación se encuentran los candidatos dentro de la red Social Twitter?
>- RQ3:     ¿Los resultadso oficiales son parecidos a los hallados en nuestro análisis?


# **II.&emsp;TEXT CLEANING**

## **2.1.&emsp; Reading dataset**
>Para este paso...

In [16]:
#!pip install emoji

In [31]:
import warnings
warnings.filterwarnings("ignore")

#from datetime import datetime, timezone
import pytz
import numpy as np              # Para operaciones matematicas
import pandas as pd             # Manejo de datos / dataframes
import re
import emoji
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from nltk.stem import PorterStemmer
%matplotlib inline

In [32]:
data_raw = '../data/raw'              # directorio que contiene toda la data sin manipular
data_processed = '../data/processed'  # directorio que contiene data procesada

In [33]:
df_lasso = pd.read_csv(data_raw+"/tweets_about_lasso.csv")
df_lasso['created_at'] = pd.to_datetime(df_lasso['created_at'], utc=pytz.UTC)
df_lasso['user.created_at'] = pd.to_datetime(df_lasso['user.created_at'], utc=pytz.UTC)
df_lasso

,created_at,favorite_count,full_text,id_str,metadata.iso_language_code,retweeted_status.created_at,retweeted_status.full_text,user.created_at,user.favourites_count,user.followers_count,user.friends_count,user.id_str,user.location,user.verified,retweet_count
0,2021-01-24 18:48:13+00:00,0,RT @Danielsalcan88: Se imaginan un gobierno de...,1353414316947550213,es,Sat Jan 23 23:24:31 +0000 2021,Se imaginan un gobierno de Guillermo Lasso con...,2020-01-05 21:38:08+00:00,22225,150,219,1213937693018607616,"Quito, Ecuador",False,409
1,2021-01-24 18:50:06+00:00,0,RT @DiegoFuentesEC: Lenín Moreno NO gobernó co...,1353414791684042757,es,Mon Jan 18 22:50:08 +0000 2021,"Lenín Moreno NO gobernó con Alianza PAIS, lo h...",2018-10-23 14:39:27+00:00,3027,228,1384,1054744113814429696,NaN,False,9
2,2021-01-24 18:51:16+00:00,0,RT @relicheandres: MITOMANÍA\r\n\r\n▪︎ Guiller...,1353415086837276673,es,Sun Jan 24 16:13:51 +0000 2021,MITOMANÍA\r\n\r\n▪︎ Guillermo Celi dice que Ma...,2012-09-21 14:11:58+00:00,109516,450,1024,837872970,"Queens, New York.",False,997
3,2021-01-24 18:52:20+00:00,0,"RT @felipeandrebsc: Carlos Vera Di la verdad, ...",1353415355914473472,es,Sun Jan 24 07:07:27 +0000 2021,"Carlos Vera Di la verdad, así reciben a tu ído...",2017-02-21 02:01:26+00:00,150546,821,591,833859141185712128,NaN,False,11
4,2021-01-24 18:53:02+00:00,0,"RT @fridayorke: Ya entendí, Hervas subió foto ...",1353415532385452032,es,Sun Jan 24 16:14:54 +0000 2021,"Ya entendí, Hervas subió foto d Lasso y Moreno...",2010-12-08 20:25:30+00:00,11385,126,989,224364373,Ecuador,False,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124064,2021-02-13 23:57:13+00:00,0,RT @Datoworld: 🇪🇨#Ecuador-Segunda Vuelta \r\n\...,1360739839604367362,es,Sat Feb 13 17:06:03 +0000 2021,🇪🇨#Ecuador-Segunda Vuelta \r\n\r\n🔵Guillermo L...,2016-07-05 16:21:13+00:00,24852,535,2200,750363914186190848,"Barquisimeto, Venezuela",False,40
124065,2021-02-13 23:57:49+00:00,0,RT @CarlosP05340289: Guillermo Lasso con su tí...,1360739987927543813,es,Sat Feb 13 17:12:41 +0000 2021,Guillermo Lasso con su título de Bachiller se ...,2014-03-31 14:53:22+00:00,14207,100,168,2449978689,NaN,False,81
124066,2021-02-13 23:58:41+00:00,0,RT @MarcoAnibal: Escuchar a Carlos Pérez Guart...,1360740207184732162,es,Sat Feb 13 13:58:09 +0000 2021,Escuchar a Carlos Pérez Guartambel en su careo...,2015-06-17 02:28:49+00:00,67781,293,777,3330587501,"Quito, Ecuador",False,308
124067,2021-02-13 23:59:12+00:00,0,RT @Guayaco__2020: - Andrés Arauz: 3'027.073\r...,1360740336100868097,es,Sat Feb 13 00:38:10 +0000 2021,- Andrés Arauz: 3'027.073\r\n- Guillermo Lasso...,2010-11-21 21:48:17+00:00,67236,4278,3499,218266485,"Guayaquil, Ecuador",False,807


## **2.2.&emsp; Removing Duplicates**
>lorem ipsum

In [34]:
# Count duplicates
len(df_lasso['full_text'])-len(df_lasso['full_text'].drop_duplicates())

104928

In [35]:
df_lasso_processed = df_lasso.drop_duplicates( ['full_text'], keep='first')

In [36]:
len(df_lasso_processed['full_text'])

19141

## **2.3.&emsp; Identify tweets by trolls**
>Lorem ipsum

In [37]:
def is_troll(Dt, Dc, Fc, fc, v):
    '''
    ----------------------------------
    | Variable Description           |
    ----------------------------------
    | Dt  stands for Date Tweet      |
    | Dc  stands for Date creation   |
    | Fc  stands for Followers count |
    | fc  stands for friend count    |
    | v   stands for verified        |
    ----------------------------------
    | Weight penalization            |
    ----------------------------------
    | Timeliness metric:  WD = 0.7   |
    | Followers metric:   WF = 0.2   |
    | Following metric:   Wf = 0.1   |
    ----------------------------------
    | Acceptance Threshold           |
    ----------------------------------
    | Non Troll   > 0.8              |
    | Troll       < 0.8              |
    ----------------------------------
    '''
    # If user is verified then is a real person
    if v:
        return True
    else:

        threshold = 0.7*((Dt-Dc).days)/180 + 0.2*(Fc/250) + 0.1*(fc/300)

        return threshold < 0.8


In [38]:
df_lasso_processed['is_troll'] = df_lasso_processed.apply(lambda x: is_troll(x['created_at'],
                                                         x['user.created_at'],
                                                         x['user.followers_count'],
                                                         x['user.friends_count'],
                                                         x['user.verified']), axis=1)

## **2.4.&emsp; Removing characters and Specific Stopwords**
>lorem ipsum

In [39]:
def clean_text(text):
    """Function to clean text extracted from the Internet with html tags
    
    Parameters
    ----------
    text: str 
      original string
    
    Return
    ------
    text: str
      modified initial string
    """

    text = text.lower() # lowercase text
    text = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", text) #Remove http links
    text = EMOJI.sub(r'', text)
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
    return text

In [40]:
# filtrado de caracteres y stopwords
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@#:,;¿?!¡-]')
EMOJI = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)

STOPWORDS = ('rt')  # Adding new stopwords to the set, 'cause theese words repeat in all docs

df_lasso_processed['full_text'] = df_lasso_processed["full_text"].apply(clean_text)

### _Luego de remover los caracteres especiales pueden existir Tweets vacios, debido a que estos solo contenian emotes o enlaces en su interior. Es por esta razon que se buscan estos espacios en blanco y se los quita_

In [41]:
df_lasso_processed[df_lasso_processed['full_text']==""]["full_text"].count()

27

In [42]:
df_lasso_processed.drop(df_lasso_processed.loc[df_lasso_processed["full_text"]==""].index, inplace=True)

In [43]:
df_lasso_processed[df_lasso_processed['full_text']==""]["full_text"].count()

0

In [44]:
df_lasso_processed.to_csv(data_processed+'/lasso_clean.csv', 
                        index=False, header=True, chunksize=10000)